# Oueslati Amine (W9GDX2)-Assignment#2-NLP

### Importing the necessary libraries 

In [ ]:
import nltk
from nltk.corpus import brown

import spacy
from spacy.tokens import Doc


nlp = spacy.load('en')
nltk.download('brown')
nltk.download('universal_tagset')

!pip install sklearn-crfsuite

import sklearn
import sklearn_crfsuite

from sklearn_crfsuite import metrics

from sklearn.model_selection import train_test_split


import ast


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
     |████████████████████████████████| 747kB 4.3MB/s 


### Getting tagged sentences with the universal tagset

In [ ]:
data = brown.tagged_sents(tagset='universal')
data

---

The zip function is used here to get the first element of each list in a sentence. 

Thus, the result will be a list of lists of tokens.

In [ ]:
words=[list(list(zip(*sent))[0]) for sent in data]
words[0]

The same methode is used here to get the list of labels, which is a list of lists of tags.

In [ ]:
labels=[list(list(zip(*sent))[1]) for sent in data]
labels[0]

---
### Feature extraction
This function is responsible for the transformation of a list of tokens to a list of dictionaries that contains the required features for the CRF model.

SpaCy is used for the feature extraction, therefore, the input list should be transformed to a spaCy document. 

knowing that the CRF algorithm needs the features of the previous, the current, and the next word to get better accuracy and efficiency; each dictionary will contain 3 sets of the same features (except for the first and the last words).


In [ ]:
def token2features(list):
    doc = Doc(nlp.vocab, words=list)
    listOfFeatures=[]
    
    for i  in range(len(doc)):
      features = {
          'lower': doc[i].lower_,
          'suffix': doc[i].suffix_,
          'prefix': doc[i].prefix_,
          'isupper': doc[i].is_upper,
          'istitle': doc[i].is_title,
          'isdigit': doc[i].is_digit
      }
      if i > 0:
          features.update({
              '-1_lower': doc[i-1].lower_,
              '-1_suffix': doc[i-1].suffix_,
              '-1_prefix': doc[i-1].prefix_,
              '-1_isupper': doc[i-1].is_upper,
              '-1_istitle': doc[i-1].is_title,
              '-1_isdigit': doc[i-1].is_digit
          })
      else:
          features['BOS'] = True

      if i < len(doc)-1:
          features.update({
              '+1_lower': doc[i+1].lower_,
              '+1_suffix': doc[i+1].suffix_,
              '+1_prefix': doc[i+1].prefix_,
              '+1_isupper': doc[i+1].is_upper,
              '+1_istitle': doc[i+1].is_title,
              '+1_isdigit': doc[i+1].is_digit,
          })
      else:
          features['EOS'] = True
      
      listOfFeatures.append(features)

    return listOfFeatures

Since the **token2features** function works for just a list of tokens ( a sentence ) , an iterative methode is required for preprocessing the whole data.

In [ ]:
listOFListsOfDictionaries  = [token2features(sent) for sent in words]

In [ ]:
listOFListsOfDictionaries[0]

### Splitting the data

The labels are already stored in the **labels** list and the text is preprocessed, so, the data can be splitted to a training set and a test set.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(listOFListsOfDictionaries, labels, test_size = 0.2, random_state = 42)

### Traning the model

In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

To evaluate the trained model, the test data set is used to predict the labels and compare it with the true ones.

In [ ]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted')

0.9784649763630752

### Pos_tagger function
This function accepts a string or a list of tokens and output the pos_tags.

The type of the input is verified, so if the input is a string it will be transformed to a list of tokens. Then the CRF model will predict the tags.

In [ ]:
def pos_tagger(sent):
  if type(sent) == str:
    doc = nlp(sent)
    for token in doc:
      tokens  = [token.text for token in doc]
  elif type(sent) == list:
    tokens = sent
  data = [token2features(tokens)]

  result = crf.predict(data)
  return result


In [ ]:
sent = input()
print("\n", pos_tagger(sent)[0])

hi

 ['NOUN']
